In [1]:
import os
import glob
import pandas as pd
import geopandas as gp
import numpy as np

In [2]:
os.getcwd()

'/home/prao/GIT/DataKind-Broadband/Notebooks'

In [3]:
data_dir = '../Data'

In [4]:
df = pd.read_csv(data_dir + '/' + 'ookla_combined_fl.csv')

In [5]:
df.head()

,quadkey,avg_d_kbps,avg_u_kbps,avg_lat_ms,tests,devices,type,quarter,year,STATEFP,COUNTYFP,TRACTCE,AFFGEOID,GEOID,NAME,LSAD,ALAND,AWATER,tile
0,320201332021332,90726,28194,35,3,2,fixed,Q1,2021,12,37,970302,1400000US12037970302,12037970302,9703.02,CT,126051207,58611667,"list(c(-85.001220703125, -84.9957275390625, -8..."
1,320231100220123,159542,31827,14,60,10,fixed,Q1,2021,12,99,905,1400000US12099000905,12099000905,9.05,CT,4601460,244009,"list(c(-80.1287841796875, -80.123291015625, -8..."
2,320231102200112,211000,52752,14,37,18,fixed,Q1,2021,12,99,5916,1400000US12099005916,12099005916,59.16,CT,3155445,0,"list(c(-80.123291015625, -80.1177978515625, -8..."
3,320231223033003,115365,20575,15,7,5,fixed,Q1,2021,12,87,971200,1400000US12087971200,12087971200,9712.00,CT,3954857,3876328,"list(c(-81.0736083984375, -81.068115234375, -8..."
4,320212301232330,217592,48549,9,21,8,fixed,Q1,2021,12,57,11906,1400000US12057011906,12057011906,119.06,CT,1560086,4190,"list(c(-82.496337890625, -82.4908447265625, -8..."


In [6]:
fcc = pd.read_csv(data_dir + '/' + 'fcc_477_census_tract_FL.csv')

In [7]:
fcc.head()

,tract,max_dn,max_up,dn10,dn100,dn250,fiber_100u,state
0,12001000200,979.959184,53.995265,1.925170,1.000000,1.000000,0.020408,12
1,12001000301,990.015000,34.653840,1.560000,1.060000,1.000000,0.000000,12
2,12001000302,989.663158,44.797558,1.178947,1.010526,1.000000,0.010526,12
3,12001000400,947.382979,33.170979,1.329787,0.946809,0.946809,0.000000,12
4,12001000500,967.066914,392.588372,2.130112,1.405204,1.364312,0.371747,12


In [8]:
### This needs to try with left outer join
df2 = pd.merge(df, fcc, left_on='GEOID', right_on='tract', how='inner')

In [9]:
df2.head()

,quadkey,avg_d_kbps,avg_u_kbps,avg_lat_ms,tests,devices,type,quarter,year,STATEFP,...,AWATER,tile,tract,max_dn,max_up,dn10,dn100,dn250,fiber_100u,state
0,320201332021332,90726,28194,35,3,2,fixed,Q1,2021,12,...,58611667,"list(c(-85.001220703125, -84.9957275390625, -8...",12037970302,854.770992,43.828244,2.78626,0.946565,0.847328,0.0,12
1,320201332021021,10732,993,24,1,1,fixed,Q1,2021,12,...,58611667,"list(c(-85.0286865234375, -85.023193359375, -8...",12037970302,854.770992,43.828244,2.78626,0.946565,0.847328,0.0,12
2,320201332023003,171894,31378,35,8,1,fixed,Q1,2021,12,...,58611667,"list(c(-85.0286865234375, -85.023193359375, -8...",12037970302,854.770992,43.828244,2.78626,0.946565,0.847328,0.0,12
3,320201332021323,131452,22349,34,6,4,fixed,Q1,2021,12,...,58611667,"list(c(-85.0067138671875, -85.001220703125, -8...",12037970302,854.770992,43.828244,2.78626,0.946565,0.847328,0.0,12
4,320201332021210,98843,8507,30,4,1,fixed,Q1,2021,12,...,58611667,"list(c(-85.023193359375, -85.0177001953125, -8...",12037970302,854.770992,43.828244,2.78626,0.946565,0.847328,0.0,12


In [10]:
df2_f = df2[df2['type'] == 'fixed']

In [11]:
# weighted avg by tests
print(f'Avg_d_kbps for 2021/FL: ' + str(np.average(df2_f['avg_d_kbps'], weights=df2_f['tests'])))
print(f'Avg_u_kbps for 2021/FL: ' + str(np.average(df2_f['avg_u_kbps'], weights=df2_f['tests'])))

Avg_d_kbps for 2021/FL: 180304.72159707
Avg_u_kbps for 2021/FL: 55861.63252433658


In [12]:
np.average(df2_f['avg_d_kbps'], weights=df2_f['devices'])

177435.91848797834

In [13]:
df2_f['avg_d_kbps'].mean()

157171.61497435038

In [14]:
wm = lambda x: np.average(x, weights=df2.loc[x.index, 'tests'])

In [15]:
df2.groupby(['GEOID','type']).agg(
    d_kbps_wm = ('avg_d_kbps', wm),
    u_kbps_wm = ('avg_u_kbps', wm)
)

d_kbps_wm     u_kbps_wm
GEOID       type                               
12001000200 fixed   144171.767327  65634.153465
            mobile   81147.314199  10703.090634
12001000301 fixed   118863.923445  10656.119617
            mobile   45877.830769  11726.815385
12001000302 fixed   152127.653061  17838.122449
...                           ...           ...
12133970301 mobile   29629.468085   6471.819149
12133970302 fixed   284073.263736  27842.417582
            mobile   42524.952542   5469.349153
12133970303 fixed    27147.843750   6073.843750
            mobile   42796.979021   6470.307692

[8186 rows x 2 columns]